<a href="https://colab.research.google.com/github/ML-DS-03/Credit-Card-Default-Classification/blob/main/Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os, sys, re, ast, csv, math, gc, random, enum, argparse, json, requests, time  
from datetime import datetime, timedelta
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) # to ensure console display all columns
pd.set_option('display.float_format', '{:0.3f}'.format)
pd.set_option('display.max_row', 40)
plt.style.use('ggplot')
from pathlib import Path
import joblib
from joblib import dump, load
from copy import deepcopy

In [3]:
!pip install pyxlsb

In [5]:
from google.colab import files
uploaded = files.upload()

Saving PoliciesFinal.xlsb to PoliciesFinal.xlsb


In [6]:
import io
data_df = pd.read_excel(io.BytesIO(uploaded['PoliciesFinal.xlsb']))

In [7]:
pip install catboost

     |████████████████████████████████| 76.6 MB 1.4 MB/s 


In [8]:
pip install pygam

     |████████████████████████████████| 1.8 MB 4.9 MB/s 


In [9]:
pip install lightgbm

In [10]:
pip install glmnet

     |████████████████████████████████| 1.6 MB 5.1 MB/s 


In [11]:
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import recall_score, roc_auc_score, classification_report, precision_recall_curve, auc, accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from glmnet import LogitNet # https://github.com/civisanalytics/python-glmnet
from pygam import LogisticGAM, s, f # https://pygam.readthedocs.io/en/latest/notebooks/tour_of_pygam.html#
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import Dataset as lgb_Dataset
from lightgbm import train as lgb_train

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

def get_lgb_classification_model(data_df, col_target, col_feature): 
    
    data_df_train, data_df_test = train_test_split(data_df, shuffle=True, random_state=100, stratify=data_df[col_target], test_size=0.2)    
    train_set = lgb_Dataset(data = data_df_train[col_feature], label = data_df_train[col_target])
    val_set = lgb_Dataset(data = data_df_test[col_feature], label = data_df_test[col_target])
    params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'n_jobs': 2,
                'learning_rate': 0.1,
                'verbose': -1,
                'seed': 100,                   
                }

    model = None
    model = lgb_train(params, train_set, 
                      num_boost_round=200, early_stopping_rounds=40, 
                      valid_sets = [train_set, val_set], verbose_eval=False)        

    return model



def get_df_woe(data_df_bin, col_bin, col_target):
    df_woe = pd.DataFrame(columns = ['feature_bin'], data = col_bin)        
    df_woe['feature'] = df_woe['feature_bin'].apply(lambda s: '_'.join(s.split('--_')[:-1]))    
    df_woe['dist_good'] = 0
    df_woe['dist_bad'] = 0    
    
    for feature in sorted(list(set(df_woe['feature']))):
        
        idx = df_woe['feature']==feature
        feature_bin_list = list(df_woe.loc[idx,'feature_bin'])
        df_woe.loc[idx,'dist_good'] = np.sum(np.array(data_df_bin[feature_bin_list])*(np.tile(np.array(data_df_bin[col_target]), (len(feature_bin_list),1)).T==0),axis=0)/np.sum(data_df_bin[col_target]==0)
        df_woe.loc[idx,'dist_bad'] = np.sum(np.array(data_df_bin[feature_bin_list])*(np.tile(np.array(data_df_bin[col_target]), (len(feature_bin_list),1)).T!=0),axis=0)/np.sum(data_df_bin[col_target]!=0)
              
    df_woe['WOE'] = np.log(df_woe['dist_good']/df_woe['dist_bad'])*100
    df_woe.loc[np.abs(df_woe['WOE'])==np.inf, 'WOE'] = 0
    df_woe.loc[df_woe['WOE'].isna(), 'WOE'] = 0

    woe_map_dict = {}
    for feature in sorted(list(set(df_woe['feature']))):
        idx = df_woe['feature']==feature
        woe_map_dict[feature] = {}
        for (feature_bin, woe) in zip(df_woe.loc[idx,'feature_bin'], df_woe.loc[idx,'WOE']):
            woe_map_dict[feature][woe] = [feature_bin.split('--_')[-1]]

    return (df_woe, woe_map_dict)



def map_woe_df(data_df_cat, woe_map_dict):
    
    def map_func(s):
        out = 0.0
        for k in woe_map_dict[col]:
            if s in woe_map_dict[col][k]:
                out = k
                break        
        return out    

    data_df_woe = data_df_cat.copy()
    
    for col in woe_map_dict:
        data_df_woe[col] = data_df_woe[col].apply(lambda s: map_func(s))
        data_df_woe[col] = data_df_woe[col].astype(np.float64)
        
    return data_df_woe



def get_model_df(data_df, col_id, col_target, col_num, col_cat, feature_method, woe_map_dict=None):

    if feature_method == 'one_hot':
        col_init = [col_id, col_target] + col_num
        model_df = data_df[col_init].copy()
        for col in col_cat:
            model_df = pd.concat([model_df, pd.get_dummies(data_df[col], prefix=f"{col}--")], axis=1)    
        col_feature = [c for c in model_df.columns if c not in [col_id, col_target]]

    if feature_method == 'woe':
        col_init = [col_id, col_target] + col_num
        
        if woe_map_dict is None:
            model_df = data_df[col_init].copy()
            for col in col_cat:
                model_df = pd.concat([model_df, pd.get_dummies(data_df[col], prefix=f"{col}--")], axis=1)    
            
            col_bin = [c for c in model_df.columns if c not in col_init]
            data_df_bin = model_df[[col_id, col_target] + col_bin].copy()
            (_, woe_map_dict) = get_df_woe(data_df_bin, col_bin, col_target)
            
        
        data_df_cat = data_df[col_cat].copy()
        data_df_woe = map_woe_df(data_df_cat, woe_map_dict)
        model_df = pd.concat([data_df[col_init], data_df_woe], axis=1) 
        col_feature = [c for c in model_df.columns if c not in [col_id, col_target]]

    return (model_df, col_feature, woe_map_dict)



def fill_na_df(data_df, col_num, col_cat, na_val_num=0, na_val_cat='NA'):
    data_df[col_num] = data_df[col_num].fillna(na_val_num)
    data_df[col_cat] = data_df[col_cat].fillna(na_val_cat)    
    return data_df


def get_undersampled_data(data_df, col_num, col_target):
    rus = RandomUnderSampler(sampling_strategy='auto', random_state=100)    
    df_rus, y_rus = rus.fit_resample(data_df[col_num], data_df[col_target])  
    df_rus[col_target] = y_rus.values
    return df_rus     


def get_oversampled_data(data_df, col_num, col_target):
    ros = RandomOverSampler(sampling_strategy='auto', random_state=100)    
    df_ros, y_ros = ros.fit_resample(data_df[col_num], data_df[col_target])  
    df_ros[col_target] = y_ros.values
    return df_ros    


def get_undersampled_oversampled_data(data_df, col_num, col_target):
    rus = RandomUnderSampler(sampling_strategy='auto', random_state=100)    
    df_rus, y_rus = rus.fit_resample(data_df[col_num], data_df[col_target])  
    df_rus[col_target] = y_rus.values    

    ros = RandomOverSampler(sampling_strategy='auto', random_state=100)    
    df_ros, y_ros = ros.fit_resample(data_df[col_num], data_df[col_target])  
    df_ros[col_target] = y_ros.values
    
    df_output = df_rus.append(df_ros)
    df_output.reset_index(drop=True, inplace=True)  
    
    return df_output    


def get_classifier_model(data_df, col_target, col_feature, model_type):
    model = None
    
    if model_type  == 'catboost':    
        model = CatBoostClassifier(
                                   random_seed=100,
                                   od_type='Iter', od_wait=20, 
                                   eval_metric='AUC', 
                                   verbose = 0,                                                                 
                                   fold_len_multiplier=2,   
                                   allow_writing_files=False,
                                   )   
 
        model.fit(data_df[col_feature], data_df[col_target])


    if model_type == 'lgb':   
        model = get_lgb_classification_model(data_df, col_target, col_feature)

    if model_type == 'glmnet':    
        model = LogitNet()
        model.fit(data_df[col_feature], data_df[col_target])

    if model_type == 'pygam':
        model = LogisticGAM(s(0) + s(1))             
        lam = np.logspace(-3, 5, 5)
        lams = [lam] * 2
        model.gridsearch(data_df[col_feature].values, data_df[col_target].values, lam=lams)


    return model


def decile_analysis(data_df, col_id, col_target, y_prob, y_true):
    decile_df = data_df[[col_id, col_target]].copy()
    decile_df['y'] = y_true
    decile_df['y_prob'] = y_prob
    
    base_response_rate = 100*decile_df[decile_df.y == 1].shape[0]/decile_df.shape[0]
    decile_df.sort_values(by = 'y_prob', inplace = True, ascending = False)
    decile_df.reset_index(inplace = True)
    decile_df['decile'] = np.nan
    d = int(np.ceil(decile_df.shape[0]/10))
    start = 0
    end = d
    
    for i in range(10):
        decile_df.loc[start:end, ['decile']] = i + 1
        start = start + d
        end = end + d
        
    decile_result_df = pd.crosstab(decile_df['decile'], decile_df['y'])
    decile_result_df.columns = ['zero', 'one']
    decile_result_df['min_prob'] = decile_df.groupby(by = ['decile']).min()['y_prob']
    decile_result_df['max_prob'] = decile_df.groupby(by = ['decile']).max()['y_prob']
    decile_result_df['count'] = decile_df.groupby(by = ['decile']).count()['y_prob']
    decile_result_df['gain'] = np.round(100*decile_result_df['one']/decile_result_df['one'].sum(), decimals = 2)
    decile_result_df['cum_gain'] = np.cumsum(decile_result_df['gain'])
    decile_result_df['lift'] = np.round((100*decile_result_df['one']/decile_result_df['count'])/base_response_rate, 2)  
    
    return decile_result_df


def get_validation_results(valid_date, model, model_type, col_id, col_target, col_num, col_cat, col_feature_init, feature_method, 
                           scaler, reducer, to_scale, woe_map_dict, print_output=False):
    
    valid_df = pd.read_csv(os.path.join(dataWorkingPath / f"model_data_{valid_date}.csv"))
    valid_df = fill_na_df(valid_df, col_num, col_cat)  
    (valid_df, _, _) = get_model_df(valid_df, col_id, col_target, col_num, col_cat, feature_method, woe_map_dict)

    for col in col_feature_init:
        if col not in valid_df.columns:
            valid_df[col] = 0
        
    if to_scale:
        valid_df[col_feature_init] = scaler.transform(valid_df[col_feature_init])        
           
    col_feature = col_feature_init.copy()
    if reducer is not None:
        col_feature_pca = [f'C{int(i)}' for i in np.arange(1,reducer.n_components_+1)]
        valid_df = pd.concat([valid_df, pd.DataFrame(reducer.transform(valid_df[col_feature].values), columns=col_feature_pca)], axis=1)
        col_feature = col_feature_pca.copy()

    if model_type in ['catboost','glmnet']:
        y_pred = model.predict(valid_df[col_feature])*1            
        y_prob = model.predict_proba(valid_df[col_feature])[:, 1]                 
    
    elif model_type in ['lgb']:
        y_prob = model.predict(valid_df[col_feature])           
        y_pred = (y_prob > 0.5)*1             
                
    elif 'pygam' in model_type.split('-'):
        y_pred = model.predict(valid_df[col_feature])*1            
        y_prob = model.predict_proba(valid_df[col_feature])  


    y_true = valid_df[col_target].values
    decile_result_df = decile_analysis(valid_df, col_id, col_target, y_prob, y_true)   
    classification_report_ = classification_report(y_true, y_pred, output_dict=True)
    result_dict = {
        f'{valid_date}_precision': np.round(classification_report_['1']['precision'], 3),
        f'{valid_date}_recall': np.round(classification_report_['1']['recall'], 3),        
        }
    for i in np.arange(1,6):
        result_dict[f"{valid_date}_top_{i}0_gain"] = np.round(decile_result_df.loc[int(i)]['cum_gain'],2)   

 
    if print_output:
        classification_report_ = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True)).transpose().reset_index() 
        print(f"## {valid_date}: {model_type}")
        print(classification_report_)
        print()
        print(decile_result_df)
        print()     

    return (classification_report_, decile_result_df, result_dict)
    

def convert_excel_date_to_date(excel_date):
    return datetime.fromordinal(datetime(1900, 1, 1).toordinal() + excel_date - 2).date()

In [12]:
feature_method_list = ['one_hot','woe']
feature_selection_method_list = ['NA','fix','vif','pca']
model_type_list = ['catboost','lgb','glmnet','pygam']
sampling_type = ['under','over','underover']

model_sampling_type_list = []
for model_type in model_type_list:
    model_sampling_type_list += [f"{model_type}-{c}" for c in sampling_type]

to_scale = True
col_cat = ['COMM_STYLE','CLAWBACK_STYLE','REGION','BENEFITCODE',
           'gender1','SMOKER1','gender2','SMOKER2','occ_class','JOINTLIFE',
           'BENESC','PRODCODE','prem_freq','product','rated']

col_num = ['initial_sum_assured','POLTERM','UWLDPERML1_MORT','UWLDPERML2_MORT','UWPERMILL1_MORT','UWPERMILL2_MORT','age_life1','age_life2','pol_tenure']
col_id = 'id'
col_target = 'target'
date_list = ['2016-06-30','2017-06-30','2018-06-30']

result_df = pd.DataFrame()

for feature_method in feature_method_list:
    
    for feature_selection_method in feature_selection_method_list:

        print(f"# feature_method: {feature_method}, feature_selection_method: {feature_selection_method}")


        for model_sampling_type in model_sampling_type_list:  
            model_type = model_sampling_type.split('-')[0]
            sampling_type = model_sampling_type.split('-')[-1]              
            print(f"model_type: {model_type}, sampling_type: {sampling_type}")

            out_dict = {
                'model_type': model_type,
                'sampling_type': sampling_type,                
                'feature_method': feature_method,
                'feature_selection_method': feature_selection_method,    
                'scale': to_scale,    
                }

            for train_date, valid_date in zip(date_list[:-1], date_list[1:]):

                train_df = pd.read_csv(os.path.join(dataWorkingPath / f"model_data_{train_date}.csv"))
                train_df = fill_na_df(train_df, col_num, col_cat)  
                
                if model_type in ['glmnet','pygam']:
                    # Apply log1p transformation to initial_sum_assured as it exhibits heavy tail
                    train_df['initial_sum_assured'] = np.log1p(train_df['initial_sum_assured'])
                    
                
                (train_df, col_feature, woe_map_dict) = get_model_df(train_df, col_id, col_target, col_num, col_cat, feature_method)
        
                if 'fix' in feature_selection_method.split('-'):   
                    col_to_keep = ['CLAWBACK_STYLE','SMOKER1','age_life1','pol_tenure']
                    col_feature = [c for c in col_feature if c.split('--')[0] in col_to_keep]
        
                if 'vif' in feature_selection_method.split('-'):   
                    col_to_rm = ['BENEFITCODE','PRODCODE','BENESC']
                    col_feature = [c for c in col_feature if c.split('--')[0] not in col_to_rm]
                    
                if 'finh' in feature_selection_method.split('-'):    
                    col_feature = ypf.get_important_features(train_df, train_df[col_target], col_feature)    
        
                scaler = None
                if to_scale:
                    scaler = StandardScaler()
                    scaler.fit(train_df[col_feature])
                    train_df[col_feature] = scaler.transform(train_df[col_feature])
        
                reducer = None
                col_feature_init = col_feature.copy()
                if 'pca' in feature_selection_method.split('-'):  
                    reducer = PCA(n_components=0.95, random_state=100)   
                    reducer.fit(train_df[col_feature].values)

                    col_feature_pca = [f'C{int(i)}' for i in np.arange(1,reducer.n_components_+1)]
                    train_df = pd.concat([train_df, pd.DataFrame(reducer.transform(train_df[col_feature].values), columns=col_feature_pca)], axis=1)
                    col_feature = col_feature_pca.copy()


                if sampling_type == 'under':
                    train_df_sampled = get_undersampled_data(train_df, col_feature, col_target)    
                    
                if sampling_type == 'over':
                    train_df_sampled = get_oversampled_data(train_df, col_feature, col_target)                    
                    
                if sampling_type == 'underover':
                    train_df_sampled = get_undersampled_oversampled_data(train_df, col_feature, col_target)                    
                    
                model = get_classifier_model(train_df_sampled, col_target, col_feature, model_type)
        

                (classification_report_, decile_result_df, result_dict) = get_validation_results(valid_date, model, model_type, col_id, col_target, col_num, col_cat, col_feature_init, 
                                                                                                 feature_method, scaler, reducer, to_scale, woe_map_dict)
                out_dict.update(result_dict)

            result_df = result_df.append(out_dict, ignore_index=True)
                 
result_df.reset_index(drop=True, inplace=True)

# feature_method: one_hot, feature_selection_method: NA
model_type: catboost, sampling_type: under


FileNotFoundError: ignored